In [1]:
%pip install git+https://github.com/denis-spe/exttorch.git
!pip install torch~=2.1.0 torch_xla[tpu]~=2.1.0 -f https://storage.googleapis.com/libtpu-releases/index.html

  Cloning https://github.com/denis-spe/exttorch.git to /tmp/pip-req-build-irir80j6
  Running command git clone --filter=blob:none --quiet https://github.com/denis-spe/exttorch.git /tmp/pip-req-build-irir80j6
  Resolved https://github.com/denis-spe/exttorch.git to commit 15ea84eaddc52a9b75de15be759a9c0f8e7cb36d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in links: https://storage.googleapis.com/libtpu-releases/index.html


**Iris**

In [ ]:
# import sys
# sys.path.append('/Volumes/Storage/DS/DL/exttorch/src/')

In [2]:
import torch
from torch import nn
from torch.optim import Adam, SGD
from exttorch.accelerator import TpuScope
from sklearn.datasets import load_iris, load_wine
from exttorch.models import Sequential
from exttorch.metrics import Precision, Recall

In [3]:
x, y = load_iris(return_X_y=True)
print(x.shape, y.shape)

(150, 4) (150,)


In [4]:
set(y)

{0, 1, 2}

In [4]:
with TpuScope():
    iris_model = Sequential([
        nn.Linear(4, 16),
        nn.ReLU(),
        nn.Linear(16, 16),
        nn.ReLU(),
        nn.Linear(16, 3)
    ])

    iris_model.compile(
        optimizer=Adam(iris_model.parameters()),
        loss=nn.CrossEntropyLoss(),
        metrics=['accuracy',
                Precision(average="weighted", zero_division=True),
                Recall(average="weighted", zero_division=True)]
        )

    history = iris_model.fit(x, y, epochs=10, validation_split=0.30)


Epoch 1/10
<class 'ValueError'> Invalid data of type <class 'torch_xla.distributed.parallel_loader.MpDeviceLoader'> for x, expected type of `np.ndarray | DataLoader | Dataset | TensorDataset` for x and np.ndarray for y <traceback object at 0x7e9e9d15a980>


ValueError: Invalid data of type <class 'torch_xla.distributed.parallel_loader.MpDeviceLoader'> for x, expected type of `np.ndarray | DataLoader | Dataset | TensorDataset` for x and np.ndarray for y

In [6]:
history.history

{}

## **Wine Dataset**

In [6]:
w_x, w_y = load_wine(return_X_y=True)
w_x.shape, w_y.shape

((178, 13), (178,))

In [7]:
set(w_y)

{0, 1, 2}

In [8]:
wine_model = Sequential([
    nn.Linear(13, 512),
    nn.ReLU(),
    nn.Linear(512, 512),
    nn.ReLU(),
    nn.Linear(512, 3)
])

wine_model.compile(
    optimizer=Adam(wine_model.parameters()),
    loss=nn.CrossEntropyLoss(),
    metrics=['accuracy',
             Precision(average="weighted", zero_division=True),
             Recall(average="weighted", zero_division=True)]
    )

wine_model.fit(w_x, w_y, epochs=10, validation_split=0.25)


Epoch 1/10


ValueError: too many values to unpack (expected 2)